In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pandas as pd

import sys
sys.path.append("../../../src/")
sys.path.append("../../")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

## Hyperparameter

In [3]:
# selected_layer = 1
# selected_filter = 47
# selected_layer = 3
# selected_filter = 20
# selected_layer = 6
# selected_filter = 19
# selected_layer = 13
# selected_filter = 112
# selected_layer = 15
# selected_filter = 148
# selected_layer = 20
# selected_filter = 398
# selected_layer = 22
# selected_filter = 485
# selected_layer = 20
# selected_filter = 398
# selected_layer = 8
# selected_filter = 99
# selected_layer = 18
# selected_filter = 101
# selected_layer = 11
# selected_filter = 75
# selected_layer = 27
# selected_filter = 161
# selected_layer = 29
# selected_filter = 334
# exp = "052532"
# num_class = 30


In [4]:
def main(exp, selected_layer, selected_filter):
    num_class = 30
    class_index = 950
    backbone = "vgg16"
    resume = "037-0"
    color_map = "nipy_spectral"
    model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    ## Load modal from dict
    
    # model
    net = model.Network(backbone=backbone, num_classes=1000, 
                        selected_layer=selected_layer)
    net.to(device)
    
    # resume from model
    resume_exp = resume.split("-")[0]
    resume_epoch = resume.split("-")[1]
    print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
    resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
    ckpt = torch.load(resume_path, map_location=device)
    net.load_state_dict(ckpt, strict=False) 
    
    summary(net, (3, 224, 224))
    
    ## Load Original Data 
    
    # Load data
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    train_transform = None
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        #transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                                 is_train=True, transform=train_transform)
    
    trainset.set_data([class_index], num_class)
    imgs_path = []                                                                                                                                                            
    images = []
    labels = []
    for img, label, img_path in trainset:                                                                                                                                     
        images.append(img.unsqueeze(0))                                                                                                                                       
        labels.append(label)                                                                                                                                                  
        imgs_path.append(img_path)  
    
    ## Create some need path
    
    save_dir = "../../../saved/pack/"
    ################### Hyper-Parameter #######################
    # exp = resume_exp
    # epoch = resume_epoch
    ##########################################################
    ab_path = os.path.join(save_dir, exp)
    
    index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
    index2image
    
    images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])
    
    # Load image
    # ext may be different.
    optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True, ext=".png")
    valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
    optimized_data_zscore = zscore(optimized_data, mean, std)
    images_zscore = zscore(images_cpu, mean, std)
    
    # Move to device
    opt_image = torch.from_numpy(optimized_data_zscore).to(device)
    original_image = torch.from_numpy(images_zscore).to(device)
    
    ## Obtain feature from conv layers.
    
    ori_activation_maps = net.get_activation_maps(original_image, 
                                                  selected_layer=selected_layer)[0]
    opt_activation_maps = net.get_activation_maps(opt_image, 
                                                  selected_layer=selected_layer)[0]
    
    images_cpu_p, optimized_data_cpu, ori_activation_maps_cpu, \
       opt_activation_maps_cpu, optimized_data_cpu_scale, \
       diff_activation_maps_cpu = \
       preprocess_arrays(images_cpu, optimized_data,
                         ori_activation_maps, opt_activation_maps,
                         selected_filter, color_map=color_map)
    concated_imgs = concat_imgs(images_cpu_p, optimized_data_cpu,
                                ori_activation_maps_cpu, opt_activation_maps_cpu,
                                optimized_data_cpu_scale, diff_activation_maps_cpu)
    
    save_path = os.path.join("../../../saved/pack/", exp, 
                             "concat.png")
    concated_imgs.save(save_path)

In [5]:
excel_path = "Single.xlsx"
excel = pd.read_excel(excel_path, dtype={"exp": str, "selected_layer": int,
                                   "selected_filter": int}).values
for data in excel:
    exp, selected_layer, selected_filter = data[0], data[1], data[2]
    exp = exp.replace("x", "")
    print(exp, selected_layer, selected_filter)
    main(exp, selected_layer, selected_filter)

052510 1 47
Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
             VGG16-3         [-1, 64, 224, 224]               0
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 73.50
Params size (MB): 0.01
Estimated Total Size (MB): 74.08
----------------------------------------------------------------
Len of new dataset is :30
Load from ../../../saved/pack/052510/35608/n07747607_10061.png
Load from ../../../saved/pack/052510/35854/n07747607_10068.png
Load from ../../../saved/pack/052510/35572/n07747607_10110.png
Load from ../../../saved/pack/052510/36064/n07747607_10134.png
Lo

=> Register fhook 6
=> Register fhook 6
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
052532 6 19
Only Keep 6th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
             VGG16-8        [-1, 128, 112, 112]               0
Total params: 112,576
Trainable params: 112,576
Non-trainable params: 0
-------------------

Len of new dataset is :30
Load from ../../../saved/pack/052519/291254/n07747607_10061.png
Load from ../../../saved/pack/052519/305387/n07747607_10068.png
Load from ../../../saved/pack/052519/278426/n07747607_10110.png
Load from ../../../saved/pack/052519/405993/n07747607_10134.png
Load from ../../../saved/pack/052519/280953/n07747607_10162.png
Load from ../../../saved/pack/052519/291290/n07747607_10166.png
Load from ../../../saved/pack/052519/224141/n07747607_10178.png
Load from ../../../saved/pack/052519/345213/n07747607_10179.png
Load from ../../../saved/pack/052519/284086/n07747607_10184.png
Load from ../../../saved/pack/052519/254870/n07747607_10192.png
Load from ../../../saved/pack/052519/290294/n07747607_10204.png
Load from ../../../saved/pack/052519/255887/n07747607_10205.png
Load from ../../../saved/pack/052519/217333/n07747607_1021.png
Load from ../../../saved/pack/052519/275591/n07747607_10213.png
Load from ../../../saved/pack/052519/288499/n07747607_10217.png
Load from ../..

=> Register fhook 15
=> Register fhook 15
052520 18 101
Only Keep 18th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256

Len of new dataset is :30
Load from ../../../saved/pack/052517/299999/n07747607_10061.png
Load from ../../../saved/pack/052517/299999/n07747607_10068.png
Load from ../../../saved/pack/052517/299999/n07747607_10110.png
Load from ../../../saved/pack/052517/299999/n07747607_10134.png
Load from ../../../saved/pack/052517/299999/n07747607_10162.png
Load from ../../../saved/pack/052517/299999/n07747607_10166.png
Load from ../../../saved/pack/052517/299999/n07747607_10178.png
Load from ../../../saved/pack/052517/299999/n07747607_10179.png
Load from ../../../saved/pack/052517/299999/n07747607_10184.png
Load from ../../../saved/pack/052517/299999/n07747607_10192.png
Load from ../../../saved/pack/052517/299999/n07747607_10204.png
Load from ../../../saved/pack/052517/299999/n07747607_10205.png
Load from ../../../saved/pack/052517/299999/n07747607_1021.png
Load from ../../../saved/pack/052517/299999/n07747607_10213.png
Load from ../../../saved/pack/052517/299999/n07747607_10217.png
Load from ../..

Len of new dataset is :30
Load from ../../../saved/pack/060380/659204/n07747607_10061.png
Load from ../../../saved/pack/060380/81035/n07747607_10068.png
Load from ../../../saved/pack/060380/64506/n07747607_10110.png
Load from ../../../saved/pack/060380/59753/n07747607_10134.png
Load from ../../../saved/pack/060380/973542/n07747607_10162.png
Load from ../../../saved/pack/060380/581588/n07747607_10166.png
Load from ../../../saved/pack/060380/930697/n07747607_10178.png
Load from ../../../saved/pack/060380/926352/n07747607_10179.png
Load from ../../../saved/pack/060380/946311/n07747607_10184.png
Load from ../../../saved/pack/060380/74869/n07747607_10192.png
Load from ../../../saved/pack/060380/65531/n07747607_10204.png
Load from ../../../saved/pack/060380/312344/n07747607_10205.png
Load from ../../../saved/pack/060380/72318/n07747607_1021.png
Load from ../../../saved/pack/060380/611719/n07747607_10213.png
Load from ../../../saved/pack/060380/647405/n07747607_10217.png
Load from ../../../sa